# **Algorithmic Trading using python**
link : https://youtu.be/xfzGZB4HhEE



**The process of quantative trading can be broken down into following steps**

1.   collect data
2.   develop hypothesis for a strategy
3.   backtest that strategy
4.   implement that strategy in production










### **API used : IEX cloud API to get financial data**
**POST : add the data to the database exposed by API(create only)**

**PUT : adds and overides the data in the database exposed by API (create     and update)**

**DELETE : delete's data from th API database**

# **Method 1: Equal-weight investing**
**1 : Equal weight is a proportional measure that gives the same importance to     each stock in a portfolio or index fund, regardless of a company's size.**


**2 : Equal weight contrasts with weighting by market capitalization, which is    more commonly used by indexes and funds.**


**3 : The concept of equally-weighted portfolios has gained interest due to  the    historical performance of small-cap stocks and the emergence of several   exchange-traded funds (ETFs).**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import math
import requests
import numpy as np
import pandas as pd
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'      #sandbox mode API token

In [ ]:
stocks = pd.read_csv('drive/MyDrive/sp_500_stocks.csv')

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

print(data)

{'avgTotalVolume': 99048697, 'calculationPrice': 'close', 'change': 3.47, 'changePercent': 0.02173, 'close': 170.85, 'closeSource': 'fiiacolf', 'closeTime': 1672813380594, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 164.178, 'delayedPriceTime': 1715016636294, 'extendedChange': 0.36, 'extendedChangePercent': 0.00217, 'extendedPrice': 170.72, 'extendedPriceTime': 1691223060028, 'high': 170.85, 'highSource': 'nme5ertil d1edye u cpai', 'highTime': 1679913135777, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 168.92, 'iexCloseTime': 1663929001558, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 163.12, 'iexOpenTime': 1671853977736, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1678331812412, 'latestPrice': 165.12, 'latestSource': 'Close', 'latestTime': 'March 18, 2022', 'latestUpdate': 1683827277587, 'latestVolume': 124635048, 'low': 161.26, 'lowSource': 'ia lmde yurpec1 5

In [ ]:
columns = ['Ticker' , 'Latest Price' , 'Market cap' , 'Number of Share to buy']
final_data = pd.DataFrame(columns = columns)
final_data

,Ticker,Latest Price,Market cap,Number of Share to buy


In [ ]:
for Ticker in stocks['Ticker'][:100]:
  api_url = f'https://sandbox.iexapis.com/stable/stock/{Ticker}/quote?token={IEX_CLOUD_API_TOKEN}'
  data_field = requests.get(api_url).json()
  final_data = final_data.append(
     pd.Series(
         [
            Ticker,
            data_field['latestPrice'],
            data_field['marketCap'],
            'NA'
         ],
         index = columns
     ),
     ignore_index = True
 )
 


In [ ]:
final_data

,Ticker,Latest Price,Market cap,Number of Share to buy
0,A,141.91,43154591065,NA
1,AAL,17.52,11301561334,NA
2,AAP,207.86,12891540358,NA
3,AAPL,164.51,2756569756425,NA
4,ABBV,165.00,294577758886,NA
...,...,...,...,...
95,CINF,132.54,21381650527,NA
96,CL,75.50,63279694487,NA
97,CLX,135.13,16609464644,NA
98,CMA,95.64,12465094778,NA


# using batch method rather than standard method
# first we will create sublist of our ticker from ticker.csv 
# then we run batch method to create our final dataframe

In [ ]:
def chunks(list , n):
  for i in range(0,len(list) , n):
    yield list[i:i+n]
chunked_list = list(chunks(stocks['Ticker'],100))

In [ ]:
chunked_string = []
for i in range(0 , len(chunked_list)):
  chunked_string.append(','.join(chunked_list[i]))

In [ ]:
columns = ['Ticker' , 'Latest Price' , 'Market cap' , 'Number of Share to buy']
stock_data = pd.DataFrame(columns = columns)
stock_data

,Ticker,Latest Price,Market cap,Number of Share to buy


In [ ]:

for value in chunked_string:
  batch_api = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={value}&types=quote&token={IEX_CLOUD_API_TOKEN}'
  data_batch = requests.get(batch_api).json()
  for symbol in data_batch:
    stock_data = stock_data.append(
        pd.Series(
            [
             symbol,
             data_batch[symbol]['quote']['latestPrice'],
             data_batch[symbol]['quote']['marketCap'],
             'NA'

            ],
            index = columns
        ),
        ignore_index = True
    )


In [ ]:
stock_data.isnull().sum()

Ticker                    0
Latest Price              0
Market cap                0
Number of Share to buy    0
dtype: int64

In [ ]:
stock_data

,Ticker,Latest Price,Market cap,Number of Share to buy
0,A,143.33,43607678824,NA
1,AAL,17.24,10952793407,NA
2,AAP,212.34,12862518281,NA
3,AAPL,167.36,2742442309115,NA
4,ABBV,164.80,289603523807,NA
...,...,...,...,...
497,YUM,123.32,36108505210,NA
498,ZBH,126.68,25873112954,NA
499,ZBRA,441.01,23702287723,NA
500,ZION,69.62,10809118975,NA


In [ ]:
print('Enter your portfolio')
try:
  val = input('Enter your limit : ')
  portfolio = int(val)
  for i in range(0 , len(stock_data.index)):
    stock_data.loc[i , 'Number of Share to buy'] = math.floor(portfolio / stock_data.loc[i , 'Latest Price'] )
    
except:
  print('that not a number')

  

Enter your portfolio
Enter your limit : 1000000


In [ ]:
stock_data

,Ticker,Latest Price,Market cap,Number of Share to buy
0,A,143.33,43607678824,6976
1,AAL,17.24,10952793407,58004
2,AAP,212.34,12862518281,4709
3,AAPL,167.36,2742442309115,5975
4,ABBV,164.80,289603523807,6067
...,...,...,...,...
497,YUM,123.32,36108505210,8108
498,ZBH,126.68,25873112954,7893
499,ZBRA,441.01,23702287723,2267
500,ZION,69.62,10809118975,14363


In [ ]:
stock_data.to_csv('stock_to_purchase.csv')